In [2]:
import os
import json
import requests

In [3]:
url = 'https://api.ebird.org/v2/data/obs/US/recent'
key = os.environ['EBIRD_API_KEY']
header = {'X-eBirdApiToken': key}
params = {'back': 1, 'includeProvisional': True}
response = requests.get(url, headers = header, params=params)


In [4]:
response

<Response [200]>

In [6]:
data = response.json()
data

[{'speciesCode': 'baleag',
  'comName': 'Bald Eagle',
  'sciName': 'Haliaeetus leucocephalus',
  'locId': 'L10439595',
  'locName': 'Jekyll Island--Driftwood Beach',
  'obsDt': '2024-01-14 17:57',
  'howMany': 2,
  'lat': 31.1034359,
  'lng': -81.4038849,
  'obsValid': True,
  'obsReviewed': False,
  'locationPrivate': False,
  'subId': 'S158924705'},
 {'speciesCode': 'grhowl',
  'comName': 'Great Horned Owl',
  'sciName': 'Bubo virginianus',
  'locId': 'L28787573',
  'locName': '17479 Hackett Rd, Dalton US-OH (40.7545,-81.6756)',
  'obsDt': '2024-01-14 17:57',
  'howMany': 1,
  'lat': 40.754453,
  'lng': -81.675586,
  'obsValid': True,
  'obsReviewed': False,
  'locationPrivate': True,
  'subId': 'S158924641'},
 {'speciesCode': 'rudtur',
  'comName': 'Ruddy Turnstone',
  'sciName': 'Arenaria interpres',
  'locId': 'L10439595',
  'locName': 'Jekyll Island--Driftwood Beach',
  'obsDt': '2024-01-14 17:57',
  'howMany': 13,
  'lat': 31.1034359,
  'lng': -81.4038849,
  'obsValid': True,
  

In [55]:
response.headers

{'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Content-Encoding': 'gzip', 'Content-Type': 'application/json;charset=utf-8', 'Date': 'Sun, 14 Jan 2024 05:02:16 GMT', 'Expires': '0', 'Pragma': 'no-cache', 'Strict-Transport-Security': 'max-age=31536000 ; includeSubDomains', 'Vary': 'Accept-Encoding, Origin, Access-Control-Request-Method, Access-Control-Request-Headers', 'X-Content-Type-Options': 'nosniff', 'X-Frame-Options': 'DENY', 'X-Xss-Protection': '1; mode=block', 'Transfer-Encoding': 'chunked'}

In [1]:
data.get()

NameError: name 'data' is not defined